<a href="https://colab.research.google.com/github/MaximeCornaton/CPE_IA-DeepLearning/blob/master/DEPOs_TP1_TP2/TP2/TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scikit-learn matplotlib

# TP2

## Exercice 2.1

## Exercice 2.2

## Exercice 2.3

## Exercice 2.4